In [6]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

Using TensorFlow backend.


In [7]:
# GRADED FUNCTION: yolo_filter_boxes

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Step 1: Compute box scores
    ### START CODE HERE ### (≈ 1 line)
    box_scores = box_confidence * box_class_probs
    ### END CODE HERE ###
    
    # Step 2: Find the box_classes thanks to the max box_scores, keep track of the corresponding score
    ### START CODE HERE ### (≈ 2 lines)
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)
    ### END CODE HERE ###
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    ### START CODE HERE ### (≈ 1 line)
    filtering_mask = box_class_scores >= threshold
    ### END CODE HERE ###
    
    # Step 4: Apply the mask to scores, boxes and classes
    ### START CODE HERE ### (≈ 3 lines)
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [8]:
# GRADED FUNCTION: iou

def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (x1, y1, x2, y2)
    box2 -- second box, list object with coordinates (x1, y1, x2, y2)
    """

    # Calculate the (y1, x1, y2, x2) coordinates of the intersection of box1 and box2. Calculate its Area.
    ### START CODE HERE ### (≈ 5 lines)
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = (xi2 - xi1) * (yi2 - yi1)
    ### END CODE HERE ###    

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    ### START CODE HERE ### (≈ 3 lines)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    ### END CODE HERE ###
    
    # compute the IoU
    ### START CODE HERE ### (≈ 1 line)
    iou = inter_area / union_area
    ### END CODE HERE ###
    
    return iou

In [9]:
# GRADED FUNCTION: yolo_non_max_suppression

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    ### START CODE HERE ### (≈ 1 line)
    nms_indices = tf.image.non_max_suppression( boxes, scores, max_boxes_tensor, iou_threshold)
    ### END CODE HERE ###
    
    # Use K.gather() to select only nms_indices from scores, boxes and classes
    ### START CODE HERE ### (≈ 3 lines)
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [10]:
# GRADED FUNCTION: yolo_eval

def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """
    
    ### START CODE HERE ### 
    
    # Retrieve outputs of the YOLO model (≈1 line)
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold (≈1 line)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with a threshold of iou_threshold (≈1 line)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [11]:
sess = K.get_session()

In [12]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)  

In [13]:
yolo_model = load_model("model_data/yolo.h5")

Instructions for updating:
keep_dims is deprecated, use keepdims instead


C:\Users\Nikhil\Miniconda3\envs\term3-gpu\lib\site-packages\keras\models.py:258: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
yolo_model.save('yolo_model.h5')

In [15]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [16]:
input_image_shape = K.placeholder(shape=(2, ))
scores, boxes, classes = yolo_eval(yolo_outputs, input_image_shape)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [39]:
from PIL import Image

In [40]:
def predict(sess, image):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the preditions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess your image
#     image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
#     image_type = imghdr.what(img_path)
#     image = Image.open(img_path)
    model_image_size = (608, 608)
    resized_image = image.resize(tuple(reversed(model_image_size)), PIL.Image.BICUBIC)
#     resized_image = cv2.resize(image,model_image_size, interpolation = cv2.INTER_CUBIC)
    image_data = np.array(resized_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    ### START CODE HERE ### (≈ 1 line)
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], 
                                                  feed_dict={yolo_model.input: image_data,
                                                            input_image_shape: [image.size[1], image.size[0]],
                                                            K.learning_phase(): 0})
    ### END CODE HERE ###

    # Print predictions info
#     print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
#     image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
#     output_image = scipy.misc.imread(os.path.join("out", image_file))
#     imshow(output_image)
    
    return out_scores, out_boxes, out_classes, image

In [18]:
# out_scores, out_boxes, out_classes = predict(sess, "sh-expo.jpg")

In [19]:
import cv2

In [49]:
def FrameCapture(path):
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    video_name = "output_video.avi"
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    video = cv2.VideoWriter(video_name, fourcc, 20.0, (1280, 720))
    while success:
        success, image = vidObj.read()
        if success:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(image)
            out_scores, out_boxes, out_classes, output_image_pil = predict(sess, image_pil)
            output_image = np.asarray(output_image_pil)
            cv2.imwrite("frames/frame%d.jpg" % count, output_image)
            video.write(output_image)
            count += 1
    video.release()

In [50]:
path = "Cars_Passing_At_Traffic_Junction.mp4"
FrameCapture(path)

car 0.64 (962, 247) (1146, 314)
car 0.69 (522, 262) (639, 321)
car 0.77 (551, 282) (751, 363)
car 0.78 (149, 282) (263, 332)
car 0.81 (526, 428) (1207, 662)
car 0.64 (962, 247) (1146, 314)
car 0.69 (522, 262) (640, 321)
car 0.77 (551, 282) (751, 363)
car 0.78 (149, 281) (263, 332)
car 0.81 (526, 428) (1207, 662)
car 0.63 (958, 251) (1147, 319)
car 0.65 (519, 268) (647, 322)
car 0.73 (558, 288) (756, 365)
car 0.74 (157, 284) (266, 336)
car 0.88 (499, 437) (1184, 667)
car 0.67 (956, 253) (1151, 323)
car 0.73 (521, 272) (654, 326)
car 0.75 (161, 287) (267, 340)
car 0.79 (589, 292) (784, 373)
car 0.86 (481, 443) (1169, 670)
car 0.68 (953, 253) (1156, 324)
car 0.68 (526, 272) (652, 328)
car 0.75 (170, 287) (278, 340)
car 0.82 (594, 295) (794, 373)
car 0.84 (463, 445) (1130, 668)
car 0.60 (950, 253) (1154, 322)
car 0.70 (552, 274) (674, 320)
car 0.76 (176, 287) (289, 338)
car 0.79 (606, 295) (810, 373)
car 0.87 (445, 446) (1111, 669)
car 0.64 (956, 255) (1160, 322)
car 0.73 (550, 273) (677, 

car 0.77 (634, 270) (780, 333)
car 0.78 (764, 256) (919, 318)
car 0.82 (977, 339) (1278, 511)
traffic light 0.61 (212, 29) (253, 60)
car 0.78 (958, 339) (1271, 505)
car 0.80 (761, 256) (924, 318)
car 0.81 (638, 269) (784, 334)
car 0.78 (641, 270) (798, 342)
car 0.79 (767, 257) (925, 320)
car 0.82 (929, 328) (1278, 498)
car 0.78 (919, 331) (1257, 492)
car 0.78 (771, 259) (925, 320)
car 0.80 (644, 271) (810, 342)
car 0.77 (899, 333) (1235, 487)
car 0.78 (775, 259) (924, 323)
car 0.81 (649, 272) (818, 342)
car 0.77 (875, 331) (1216, 483)
car 0.78 (773, 259) (925, 324)
car 0.82 (650, 273) (827, 346)
car 0.75 (776, 259) (928, 327)
car 0.77 (649, 273) (827, 346)
car 0.82 (857, 330) (1193, 480)
car 0.76 (781, 260) (929, 327)
car 0.78 (672, 275) (833, 348)
car 0.82 (844, 327) (1163, 474)
car 0.76 (782, 260) (922, 327)
car 0.79 (828, 325) (1122, 471)
car 0.79 (673, 277) (840, 349)
traffic light 0.62 (219, 45) (255, 76)
car 0.72 (787, 259) (917, 327)
car 0.79 (815, 324) (1115, 472)
car 0.81 (672

car 0.67 (1183, 239) (1280, 316)
car 0.82 (916, 326) (1279, 497)
car 0.65 (1178, 233) (1278, 315)
car 0.68 (625, 237) (767, 321)
car 0.84 (904, 321) (1280, 492)
car 0.73 (631, 235) (774, 321)
car 0.86 (894, 317) (1279, 490)
car 0.61 (1181, 237) (1275, 310)
car 0.86 (857, 311) (1254, 476)
car 0.66 (645, 233) (786, 318)
car 0.86 (838, 306) (1239, 473)
car 0.71 (649, 234) (798, 321)
car 0.86 (824, 305) (1219, 469)
car 0.75 (655, 233) (806, 321)
car 0.84 (810, 305) (1181, 460)
car 0.62 (288, 220) (370, 274)
car 0.68 (663, 232) (807, 322)
car 0.86 (794, 304) (1167, 459)
car 0.82 (777, 294) (1140, 455)
car 0.71 (680, 230) (842, 322)
car 0.84 (761, 299) (1115, 447)
car 0.66 (683, 230) (851, 323)
car 0.86 (748, 299) (1098, 441)
car 0.64 (296, 223) (386, 273)
car 0.66 (682, 231) (852, 323)
car 0.83 (724, 298) (1081, 441)
car 0.60 (294, 225) (388, 276)
car 0.68 (693, 235) (851, 321)
car 0.87 (720, 298) (1056, 437)
car 0.69 (700, 232) (852, 317)
car 0.72 (291, 224) (392, 273)
car 0.86 (698, 289) 